In [11]:
# Using Bert States obtained from Google Colab

In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
import regression_paper
import models

In [2]:
reg_obj = regression_paper.Gutenberg_Emotion()
df = reg_obj.get_corpus(regression_paper.EMOBANK_PATH)

In [3]:
features = None
with open('bert_states_10000.pkl', 'rb') as f:
  features = pickle.load(f)

In [4]:
print(len(features), features[0].shape)

100 (100, 768)


In [5]:
# Each of the 100 vectors further have 100 vectors 
# (as 100 vectors were processed in a group instead of 1000)
# Due to memory constraints

In [6]:
# In 1-D format
all_features = list()
for i in range(100):
  for j in range(100):
    if i > 0 and j==100:
      continue
    else:
      all_features.append(features[i][j])
features = all_features

print(len(features), len(features[0]))

10000 768


In [9]:
lt = "A"
labels = df[lt][:10000]

# labels = [df[lt].tolist()[i+1:i+100] for i in range(0, 10000, 100)]
# labels[0].insert(0, df[lt].tolist()[0])

# # Flatten again
# labels = sum(labels, [])
# print(len(labels))

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.1)
print(len(train_features), len(test_features))

9000 1000


In [10]:
model_obj = models.Models()
svr = model_obj.svr(train_features, train_labels, test_features, test_labels)
print(svr)

{'model': Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svr',
                 SVR(C=0.8, cache_size=200, coef0=0.0, degree=3, epsilon=0.2,
                     gamma='scale', kernel='rbf', max_iter=-1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False), 'score_train': 0.6197145886598122, 'score_val': 0.3351597215142541, 'rmse_val': 0.2174676998428308, 'rmse_train': 0.15946283769480094}


In [11]:
# For the book
book_info = reg_obj.get_book(11)
book = book_info["text"]
train_df = pd.DataFrame(reg_obj.book_formatting(book), columns=["text"])

train_df = reg_obj.split_text_read_time(train_df, 
                                    reading_time=10, 
                                    in_seconds=True)

train_df.to_csv("11_train.csv")

In [12]:
with open('book_11_600_2.pkl', 'rb') as f:
  book_features = pickle.load(f)

In [14]:
# In 1-D format
book_all_features = list()
for i in range(6):
  for j in range(100):
    if i == 5 and j == 60:
        break
    else:
      book_all_features.append(book_features[i][j])
book_features = book_all_features

print(len(book_features), len(book_features[0]))

560 768


In [15]:
# # Remove 100, 200, etc.
# sentences = [train_df["text"].tolist()[i+1:i+100] for i in range(0, 600, 100)]
# sentences[0].insert(0, train_df["text"].tolist()[0])

# # Flatten again
# sentences = sum(sentences, [])
# print(len(sentences))

In [16]:
# train_df = pd.DataFrame(sentences, columns=["text"])
train_df

,text
0,Alice’s Adventures in Wonderland\n\nby Lewis C...
1,The Rabbit Sends in a Little Bill\n CHAPTER V....
2,The Mock Turtle’s Story\n CHAPTER X. The Lobst...
3,There was nothing so _very_ remarkable in that...
4,I shall be late!” (when she thought it over af...
...,...
555,"But her sister sat still just as she left her,..."
556,The long grass rustled at her feet as the Whit...
557,"So she sat on, with closed eyes, and half beli..."
558,"Lastly, she pictured to herself how this same ..."


In [17]:
book_predict = svr["model"].predict(book_features)
book_predict_df = pd.DataFrame(book_predict)
print(book_predict_df.describe())

                0
count  560.000000
mean     3.150582
std      0.144414
min      2.792654
25%      3.048765
50%      3.119227
75%      3.231957
max      3.919521


In [254]:
book_predict_df.to_csv("1777_D.csv")